In [55]:
import matplotlib.pyplot as plt
import numpy as np
import GenNumPseudoAleat as rnd
import math
%matplotlib inline

#Muestra las tablas con la informacion inicial
def infoInicial(Llegadas,Servicio):
    print "\nINFORMACION INICIAL"
    print "Iteracion\tLlegadas\tServicio(min)"
    for i in range(len(Llegadas)):
        print "[",i,"]\t\t",int(Llegadas[i]),"\t\t",int(Servicio[i])
        
#Muestra las tablas con la informacion de cada iteracion de la simulacion
def infoSimulacion(Simulacion):
    Llegadas = Simulacion[0]
    Servicio = Simulacion[1]
    Cola = Simulacion[2]
    Sistema = Simulacion[3]
    Salidas = Simulacion[4]
    print "\nINFORMACION SIMULADA"
    print "Iteracion\tLlegadas\tServicio\tCola\t\tSistema\t\tSalidas"
    for i in range(len(Llegadas)):
        print "[",i,"]\t\t",int(Llegadas[i]),"\t\t",int(Servicio[i]),"\t\t",int(Cola[i]),"\t\t",int(Sistema[i]),"\t\t",int(Salidas[i])

#Simula el comportamiento del sistema al inicio de cada iteracion
def RunSimulation(Llegadas,Servicio,time,N):
    Cola = [0.0] * N
    Sistema = [0.0] * N
    Salidas = [0.0] * N
    TdispS = [time] * N
    NServ = [0.0] * N
    for i in range(N):
        if(i==0):
            Sistema[i] = Llegadas[i]
            NServ[i] = math.ceil(TdispS[i]/float(Servicio[i]))
        else:
            if(TdispS[i-1]/float(Servicio[i-1]) < Llegadas[i-1]):
                TdispS[i] = TdispS[i-1] - (NServ[i-1]*Servicio[i-1]) + time
            NServ[i] = math.ceil(TdispS[i]/float(Servicio[i]))
            if(time/float(Servicio[i-1]) < Llegadas[i-1] + Cola[i-1]):
                Cola[i] = Llegadas[i-1] + Cola[i-1] - NServ[i-1]
            Sistema[i] = Cola[i] + Llegadas[i]
            if(time/float(Servicio[i-1]) > Llegadas[i-1]+Cola[i-1]):
                Salidas[i] = Salidas[i-1] + Llegadas[i-1] + Cola[i-1]
            else:
                if(TdispS[i]==time):
                    Salidas[i] = Salidas[i-1] + NServ[i-1]
                else:
                    Salidas[i] = Salidas[i-1] + NServ[i-1] - 1
        #print "[",i,"] Lleg: ",int(Llegadas[i])," Serv: ",int(NServ[i])," TDispS: ",int(TdispS[i])," Cola: ",int(Cola[i])," Sis: ",int(Sistema[i])," Sali: ",int(Salidas[i])
    Simulacion = [Llegadas,NServ,Cola,Sistema,Salidas]
    return Simulacion

#Calcula promedios de la simulacion
def infoSistem(Simulacion,time):
    Llegadas = Simulacion[0]
    Servicio = Simulacion[1]
    Cola = Simulacion[2]
    Sistema = Simulacion[3]
    promLleg = 0
    promServ = 0
    promCola = 0
    promSistem = 0 
    for i in range(len(Llegadas)):
        promLleg = promLleg + Llegadas[i]
        promServ = promServ + Servicio[i]
        promCola = promCola + Cola[i]
        promSistem = promSistem + Sistema[i]
    timeCola = CalcTiempCola(Llegadas,Servicio,time)
    timeSistem = CalcTiempSistema(timeCola,Servicio,time)
    promTimeCola = 0
    promTimeSistem = 0
    for i in range(len(timeCola)):
        promTimeCola = promTimeCola + timeCola[i]
        promTimeSistem = promTimeSistem + timeSistem[i]
    print "\nINFORMACION DEL SISTEMA"
    print "Clientes promedio en Llegar: ",int(round(promLleg/len(Llegadas)))
    print "Tiempo entre Llegadas: ",round(time/(promLleg/len(Llegadas)),2)
    print "Clientes promedio en ser Atendidos: ",int(round(promServ/len(Servicio)))
    print "Tiempo de Servicio: ",round(time/(promServ/len(Servicio)),2)
    print "Clientes promedio en la Cola: ",int(round(promCola/len(Cola)))
    print "Tiempo en la Cola: ",round(promTimeCola/len(timeCola),2)
    print "Clientes promedio en el Sistema: ",int(round(promSistem/len(Sistema)))
    print "Tiempo en la Sistema: ",round(promTimeSistem/len(timeSistem),2)

#Calcula el tiempo de cada cliente en la cola
def CalcTiempCola(Llegadas,Servicio,time):
    atendidos = 0
    for i in range(len(Servicio)):
        atendidos = int(atendidos + Servicio[i])
    TServ = [1.0] * atendidos
    TLleg = [1.0] * atendidos
    cont1 = 0
    cont2 = 0
    for i in range(len(Servicio)):
        if(cont1<atendidos):
            for j in range(int(Servicio[i])):
                TServ[cont1+j] = i
            cont1 = cont1+int(Servicio[i])
        if(cont2<atendidos):
            for j in range(int(Llegadas[i])):
                TLleg[cont2+j] = i
            cont2 = cont2+int(Llegadas[i])
    TimeCola = [0.0] * atendidos
    for i in range(atendidos):
        if(TServ[i]-TLleg[i] > 0):
            TimeCola[i] = (TServ[i] - TLleg[i])*time
    return TimeCola
    
#Calcula el tiempo de cada cliente en el sistema
def CalcTiempSistema(TimeCola,Servicio,time):
    TServ = [1.0] * len(TimeCola)
    cont = 0
    for i in range(len(Servicio)):
        if(cont<len(TimeCola)):
            for j in range(int(Servicio[i])):
                TServ[cont+j] = time/float(Servicio[i])
            cont = cont+int(Servicio[i])
    TimeSistem = [1.0] * int(len(TimeCola))
    for i in range(len(TimeSistem)):
        TimeSistem[i] = TimeCola[i] + TServ[i]
    return TimeSistem
    
def main():  
    ly = 4 #Tasa Llegada (Clientes x Intervalo de Tiempo)
    miu = 3 #Tasa Servicio (Clientes x Intervalo de Tiempo)
    time = 60 #Intervalo de Tiempo (Minutos)
    N = 8 #Iteraciones (Repeticiones x Intervalo de Tiempo)

    Llegadas = [ly] * N
    Servicio = [time/float(miu)] * N
    infoInicial(Llegadas,Servicio)
    
    Simulacion = RunSimulation(Llegadas,Servicio,time,N)
    infoSimulacion(Simulacion)
    
    infoSistem(Simulacion,time)

if __name__ == "__main__":
    main()


INFORMACION INICIAL
Iteracion	Llegadas	Servicio(min)
[ 0 ]		4 		20
[ 1 ]		4 		20
[ 2 ]		4 		20
[ 3 ]		4 		20
[ 4 ]		4 		20
[ 5 ]		4 		20
[ 6 ]		4 		20
[ 7 ]		4 		20

INFORMACION SIMULADA
Iteracion	Llegadas	Servicio	Cola		Sistema		Salidas
[ 0 ]		4 		3 		0 		4 		0
[ 1 ]		4 		3 		1 		5 		3
[ 2 ]		4 		3 		2 		6 		6
[ 3 ]		4 		3 		3 		7 		9
[ 4 ]		4 		3 		4 		8 		12
[ 5 ]		4 		3 		5 		9 		15
[ 6 ]		4 		3 		6 		10 		18
[ 7 ]		4 		3 		7 		11 		21

INFORMACION DEL SISTEMA
Clientes promedio en Llegar:  4
Tiempo entre Llegadas:  15.0
Clientes promedio en ser Atendidos:  3
Tiempo de Servicio:  20.0
Clientes promedio en la Cola:  4
Tiempo en la Cola:  60.0
Clientes promedio en el Sistema:  8
Tiempo en la Sistema:  80.0
